# ETL (Extract Transform Load)

Extraer, Transformar, Carga <div>
Es un proceso que se utiliza para mover datos de una fuente (Extract), realizar modificaciones en esos datos según sea necesario (Transform), y cargar los datos resultantes en un destino deseado (Load). 

***En este archivo van todas las transformaciones requeridas***

## 3. Preparación de los datos

### 3.1 Importamos  librerías

In [93]:
import pandas as pd  # type: ignore
import numpy as np # type: ignore
import seaborn as sns  # type: ignore
import matplotlib.pyplot as plt # type: ignore
import ast
import scipy.stats as stats
import warnings 
warnings.filterwarnings('ignore')

### 3.2 Carga inicial de los datos

In [94]:
# Cargar el archivo 
games = pd.read_json("Data/output_steam_games.json", lines=True)

In [95]:
# Convertimos en Dataframe
df_games = pd.DataFrame(games)

### 3.3 Preparación de los datos 

***Gracias al EDA ya conocemos nuestros datos, comenzaremos eliminando los datos faltantes.*** 
***Archivo games*** 

In [96]:
df_games = df_games.dropna(how='all')
print(df_games.shape)

(32135, 13)


In [97]:
#Observemos los valores no numericos en la columna 'price'
mask_filter = pd.to_numeric(df_games['price'], errors='coerce').isna()
no_numeric_values = df_games.loc[mask_filter, 'price'].unique()
print(no_numeric_values)

['Free To Play' 'Free to Play' None 'Free' 'Free Demo' 'Play for Free!'
 'Install Now' 'Play WARMACHINE: Tactics Demo' 'Free Mod' 'Install Theme'
 'Third-party' 'Play Now' 'Free HITMAN™ Holiday Pack' 'Play the Demo'
 'Starting at $499.00' 'Starting at $449.00' 'Free to Try' 'Free Movie'
 'Free to Use']


In [98]:
#convertimos los valores no numericos a 0 (cero) para representar a los juegos gratis 
df_games['price'] = pd.to_numeric(df_games['price'], errors= 'coerce')
df_games['price'].fillna(0, inplace=True)

In [99]:
df_games['price'].astype('float16')

88310     4.988281
88311     0.000000
88312     0.000000
88313     0.990234
88314     2.990234
            ...   
120440    1.990234
120441    4.988281
120442    1.990234
120443    4.988281
120444    4.988281
Name: price, Length: 32135, dtype: float16

In [100]:
# Tratamiento de valores atipicos o outliers
# Buscamos el 1er cuartil y 3er cuartil
q1, q3 = np.percentile(df_games['price'], [25, 75])
print("1er cuartil = ", q1)
print("3er cuartil = ", q3)

# Buscamos la diferencia
iqr = q3 - q1
print("IQR es la diferencia entre el 3er y 1er cuartil = ",iqr)

# Limite inferior y superior 
limite_inferior = q1 - (1,5 * iqr)
limite_superior = q3 + (1,5 * iqr)
print("Limite Inferior = ", limite_inferior)
print("Limite superior = ", limite_superior)

1er cuartil =  1.99
3er cuartil =  9.99
IQR es la diferencia entre el 3er y 1er cuartil =  8.0
Limite Inferior =  [  0.99 -38.01]
Limite superior =  [10.99 49.99]


In [101]:
# Usaremos z_score para el tratamiento de valores atípicos.
df_games['Zscore_price'] = stats.zscore(df_games['price'])
# Detectar valores atípicos (z_score)
df_games[(df_games['Zscore_price'] < -3) | (df_games['Zscore_price']> 3)]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,Zscore_price
88328,Poppermost Productions,"[Free to Play, Indie, Simulation, Sports]",SNOW - Lifetime Pack,SNOW - Lifetime Pack,http://store.steampowered.com/app/774279/SNOW_...,2018-01-03,"[Free to Play, Indie, Simulation, Sports]",http://steamcommunity.com/app/774279/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",59.99,0.0,774279.0,Poppermost Productions,3.288407
88735,SEGA,"[Action, Adventure, Casual, RPG]",SEGA Mega Drive and Genesis Classics,SEGA Mega Drive and Genesis Classics,http://store.steampowered.com/app/34270/SEGA_M...,2010-06-01,"[Action, RPG, Casual, Adventure, Retro, Classic]",http://steamcommunity.com/app/34270/reviews/?b...,"[Single-player, Multi-player, Co-op, Local Co-...",61.99,0.0,34270.0,D3T Limited,3.416104
89356,Square Enix,"[Action, Adventure]",Sleeping Dogs,Sleeping Dogs,http://store.steampowered.com/app/202170/Sleep...,None,"[Open World, Action, Martial Arts, Third Perso...",http://steamcommunity.com/app/202170/reviews/?...,"[Single-player, Steam Achievements, Full contr...",74.76,0.0,202170.0,United Front Games,4.231452
89404,Pilgway,[Animation &amp; Modeling],3D-Coat V4.8,3D-Coat V4.8,http://store.steampowered.com/app/100980/3DCoa...,2012-10-02,[Animation & Modeling],http://steamcommunity.com/app/100980/reviews/?...,[Steam Cloud],99.99,0.0,100980.0,Pilgway,5.842355
89488,Activision,[Action],Call of Duty®: Black Ops II,Call of Duty®: Black Ops II,http://store.steampowered.com/app/202970/Call_...,2012-11-12,"[Action, Multiplayer, FPS, Shooter, First-Pers...",http://steamcommunity.com/app/202970/reviews/?...,"[Single-player, Multi-player, Co-op, Steam Ach...",59.99,0.0,202970.0,Treyarch,3.288407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118334,SEGA,[Strategy],Total War™: ROME II - Emperor Edition,Total War™: ROME II - Emperor Edition,http://store.steampowered.com/app/214950/Total...,2013-09-02,"[Strategy, Historical, Turn-Based Strategy, Gr...",http://steamcommunity.com/app/214950/reviews/?...,"[Single-player, Multi-player, Steam Trading Ca...",59.95,0.0,214950.0,Creative Assembly,3.285853
118628,Gaijin Entertainment,"[Action, Free to Play, Massively Multiplayer, ...",Star Conflict: Mercenary Pack - Elite Pilot,Star Conflict: Mercenary Pack - Elite Pilot,http://store.steampowered.com/app/222282/Star_...,2013-02-27,"[Action, Free to Play, Massively Multiplayer, ...",http://steamcommunity.com/app/222282/reviews/?...,"[Multi-player, MMO, Downloadable Content, Part...",79.99,0.0,222282.0,Star Gem Inc.,4.565381
118663,"Sector3 Studios, RaceRoom Entertainment AG","[Free to Play, Racing, Simulation, Sports]",RaceRoom Racing Experience,RaceRoom Racing Experience,http://store.steampowered.com/app/211500/RaceR...,2013-02-12,"[Racing, Free to Play, Multiplayer, Simulation...",http://steamcommunity.com/app/211500/reviews/?...,"[Single-player, Multi-player, Steam Trading Ca...",160.91,0.0,211500.0,Sector3 Studios,9.732019
118761,Nevercenter Ltd. Co.,[Animation &amp; Modeling],Silo 2,Silo 2,http://store.steampowered.com/app/100400/Silo_2/,2012-12-19,"[Animation & Modeling, Software]",http://steamcommunity.com/app/100400/reviews/?...,None,99.99,0.0,100400.0,Nevercenter Ltd. Co.,5.842355


In [102]:
# Eliminar valor atípico
df_games = df_games[(df_games['Zscore_price'] > -3) & (df_games['Zscore_price'] < 3)]

Ya transformamos la columna precio, ya eliminamos los outliers y los valores faltantes, seguimos !

In [103]:
# Ahora transformaremos la columna release_date a tipo to_datetime para mejor manipulacion 
df_games['release_date'] = pd.to_datetime(df_games['release_date'], format= '%Y-%m-%d', errors='coerce')

In [104]:
# Creamos una columna nueva con esos valores, pero solo extraeremos el año y cambiamos el tipo de dato
df_games['Year'] = df_games['release_date'].dt.year.astype('Int64')

In [105]:
# Buscamos cuales son los valores faltantes, ademas cuantos de esos valores existen 
mask_filter = df_games['Year'].isna()
diferent_Value = df_games.loc[mask_filter, 'Year'].unique()
print(f"Valores unicos faltantes {diferent_Value}, Numero de valores faltantes {mask_filter.sum()}")

Valores unicos faltantes <IntegerArray>
[<NA>]
Length: 1, dtype: Int64, Numero de valores faltantes 2334


In [106]:
# LLenamos los valores NAN con la mediana
def llenar_nan(df, columna):
    mediana = df_games['Year'].median()
    df[columna] = df[columna].fillna(mediana)
    return df 

df_games = llenar_nan(df_games, 'Year')

In [107]:
def null(df, decimales=2):
    """
    Vamos a observar cuantos valores falatantes hay en el dataset
    """
    df_nulos = pd.DataFrame({
        "Numeros de nulos" : df.isnull().sum(),
        "Porcentaje de nulos" : (df.isnull().sum() / df.shape[0]) * 100.0
    })
    df_nulos['Porcentaje de nulos'] = df_nulos['Porcentaje de nulos'].round(decimales).astype(str) + "%"
    return df_nulos

In [108]:
null(df_games, decimales= 2)

,Numeros de nulos,Porcentaje de nulos
publisher,8001,25.13%
genres,3264,10.25%
app_name,2,0.01%
title,2036,6.4%
url,0,0.0%
release_date,2334,7.33%
tags,161,0.51%
reviews_url,2,0.01%
specs,612,1.92%
price,0,0.0%


In [109]:
# Eliminaremos las columnas no necesarias
df_games = df_games.drop(columns=['release_date', 'Zscore_price', 'early_access', 'title'])

In [110]:
# Eliminaremos filas con valores faltantes muy bajos
df_games = df_games.dropna(subset=['id'])
df_games = df_games.dropna(subset=['reviews_url'])
df_games = df_games.dropna(subset=['app_name'])

Como sabes tenemos valores faltantes en cada columna anidada:
+ genres hay 3264
+ tags hay 161
+ specs hay 612

Podemos aplicar algunas tecnicas de imputacion de valores faltantes, pero teniendo en cuenta que el proyecto es un MLops, optaremos por la eliminacion de los datos faltantes. 

In [111]:
# Eliminammos las filas con valores faltantes en la columna genres
df_games = df_games.dropna(subset=['genres'])
# Ahora eliminamos las filas con valores faltantes en la columna specs
df_games = df_games.dropna(subset=['specs'])
# finalmente eliminamos las filas con valores faltantes en la columna tags
df_games = df_games.dropna(subset=['tags'])

Antes de continuar, vamos a eliminar la columna Publisher y completar las filas con valores faltantes con el valor 'SD' que significa sin datos para la columna developer.

In [112]:
# Eliminamos la columna publisher
df_games = df_games.drop(columns=['publisher'])

In [113]:
# Remplazamos los valores faltantes (NAN) con SD
df_games['developer'].replace(['', None], 'SD', inplace=True)

In [114]:
# Verifiquemos si todavia hay valores faltantes
null(df_games, decimales=2)

,Numeros de nulos,Porcentaje de nulos
genres,0,0.0%
app_name,0,0.0%
url,0,0.0%
tags,0,0.0%
reviews_url,0,0.0%
specs,0,0.0%
price,0,0.0%
id,0,0.0%
developer,0,0.0%
Year,0,0.0%


In [115]:
# Veamos cuantas registros tenemos antes de desanidar.
df_games.shape[0]

28316

In [116]:
# Veamos la columna anidada, antes del cambio 
filtro  = df_games['genres'].value_counts()
filtro

genres
[Action]                                                1853
[Action, Indie]                                         1648
[Simulation]                                            1382
[Casual, Simulation]                                    1359
[Action, Adventure, Indie]                              1080
                                                        ... 
[Action, Adventure, Free to Play, RPG, Early Access]       1
[Adventure, Free to Play, Indie, RPG, Strategy]            1
[Action, Casual, Indie, RPG, Sports]                       1
[Action, Casual, RPG, Strategy]                            1
[Adventure, Casual, RPG, Simulation, Early Access]         1
Name: count, Length: 838, dtype: int64

In [117]:
# Veamos la columna anidada, antes del cambio 
filtro_specs = df_games['specs'].value_counts()
filtro_specs

specs
[Single-player]                                                                                                                                                                                                                                         2760
[Single-player, Downloadable Content]                                                                                                                                                                                                                   1183
[Single-player, Shared/Split Screen, Downloadable Content, Steam Achievements, Steam Trading Cards, Partial Controller Support, Steam Cloud, Steam Leaderboards]                                                                                        1015
[Single-player, Steam Achievements]                                                                                                                                                                                                        

In [118]:
# Veamos la columna anidada, antes del cambio 
filtro_tags = df_games['tags'].value_counts()
filtro_tags

tags
[Casual, Simulation]                                                         1289
[Action]                                                                     1098
[Simulation]                                                                 1074
[Strategy, RPG, Indie]                                                        630
[Strategy]                                                                    621
                                                                             ... 
[Indie, Casual, Puzzle, 2D, Family Friendly]                                    1
[Early Access, Simulation, Strategy, Indie, Sports, e-sports, Management]       1
[Racing, Multiplayer, Action]                                                   1
[Action, Casual, Indie, Arcade, 2D, Top-Down]                                   1
[Indie, Casual, Puzzle, Singleplayer, Atmospheric, Relaxing]                    1
Name: count, Length: 13421, dtype: int64

In [119]:
# Verifica el número de valores no nulos en cada columna
columnas_a_verificar = ['genres', 'tags', 'specs']
for columna in columnas_a_verificar:
  numero_no_nulos = df_games[columna].notnull().sum()
  print(f'La columna {columna} tiene {numero_no_nulos} valores no nulos.')

# Comprueba si todos los valores son iguales
if df_games[columnas_a_verificar].notnull().sum().all():
  print('Las columnas genres, tags y specs tienen la misma cantidad de filas.')
else:
  print('Las columnas genres, tags y specs NO tienen la misma cantidad de filas.')


La columna genres tiene 28316 valores no nulos.
La columna tags tiene 28316 valores no nulos.
La columna specs tiene 28316 valores no nulos.
Las columnas genres, tags y specs tienen la misma cantidad de filas.


In [120]:
df_genres = df_games[['id', 'genres']].explode('genres')
df_tags = df_games[['id', 'tags']].explode('tags')
df_specs = df_games[['id', 'specs']].explode('specs')

In [121]:
# Veamos cada columna como quedo en numero de filas 
print("tenemmos estas filas en la columna genres:", df_genres.shape[0])
print("tenemmos estas filas en la columna tags:",df_tags.shape[0])
print("tenemmos estas filas en la columna specs:",df_specs.shape[0])

tenemmos estas filas en la columna genres: 70317
tenemmos estas filas en la columna tags: 152852
tenemmos estas filas en la columna specs: 126551


Cada columna luego de desanidar vemos que tienen diferentes longitudes, ahora vamos a unir esas columnas con el dataset general con el metodo merge.

In [122]:
# Combinamos genres con el datasets general 
df_temp = pd.merge(df_games.drop(columns=['genres', 'tags', 'specs']), df_genres, on='id', how='outer')
# Combina df_temp con df_tags
df_temp = pd.merge(df_temp, df_tags, on='id', how='outer')
# Finalmente combina df_temp con df_specs
df_combined = pd.merge(df_temp, df_specs, on='id', how='outer')

In [123]:
df_combined.sample(5)

,app_name,url,reviews_url,price,id,developer,Year,genres,tags,specs
693245,Demise of Nations,http://store.steampowered.com/app/338810/Demis...,http://steamcommunity.com/app/338810/reviews/?...,0.00,338810.0,Noble Master LLC,2017,Free to Play,Free to Play,Online Multi-Player
279219,A Sirius Game,http://store.steampowered.com/app/392930/A_Sir...,http://steamcommunity.com/app/392930/reviews/?...,4.99,392930.0,"Overcloud9, Phantasm Games",2015,RPG,Casual,Single-player
933499,JumpFist,http://store.steampowered.com/app/707470/JumpF...,http://steamcommunity.com/app/707470/reviews/?...,4.99,707470.0,BoomCat Studios,2017,Casual,Indie,Single-player
883605,Sword Art Online: Hollow Realization Deluxe Ed...,http://store.steampowered.com/app/607890/Sword...,http://steamcommunity.com/app/607890/reviews/?...,49.99,607890.0,AQURIA,2017,Adventure,Nudity,Steam Achievements
33012,Call of Duty®: Modern Warfare® 3,http://store.steampowered.com/app/115300/Call_...,http://steamcommunity.com/app/115300/reviews/?...,39.99,115300.0,"Infinity Ward,Sledgehammer Games,Aspyr (Mac)",2011,Action,Adventure,Steam Achievements


In [124]:
# Veamos cuantas registros tenemos despues de desanidar.
df_combined.shape[0]

1977302

Tenemos una total de casi 2 millones de registros despues de desanidar, veamos si hay valores faltantes

In [125]:
null(df_combined, decimales=2)

,Numeros de nulos,Porcentaje de nulos
app_name,0,0.0%
url,0,0.0%
reviews_url,0,0.0%
price,0,0.0%
id,0,0.0%
developer,0,0.0%
Year,0,0.0%
genres,0,0.0%
tags,0,0.0%
specs,0,0.0%


In [126]:
def suma_duplicados(df):
    df_duplicados = pd.DataFrame({
        "Numero de Duplicados" :  df[df.duplicated()].sum(),
        "Porcentaje de Duplicados" : (df[df.duplicated()].sum() / df.shape[0]) * 100.0
    })
    df_duplicados["Porcentaje de Duplicados"] = df_duplicados["Porcentaje de Duplicados"].astype(str) + "%"
    return df_duplicados

In [127]:
suma_duplicados(df_combined)

,Numero de Duplicados,Porcentaje de Duplicados
app_name,0,0.0%
url,0,0.0%
reviews_url,0,0.0%
price,0.0,0.0%
id,0.0,0.0%
developer,0,0.0%
Year,0,0.0%
genres,0,0.0%
tags,0,0.0%
specs,0,0.0%


Como ultimos cambios para ya tener el dataset listo para las consultas, vamos a cambiar el tipo de dato y algunas columnas el nombre 

In [128]:
#Cambiamos los nombres de las columnas 
df_combined.rename(columns={'app_name' : 'name', 'id': 'item_id' }, inplace=True)

In [129]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1977302 entries, 0 to 1977301
Data columns (total 10 columns):
 #   Column       Dtype  
---  ------       -----  
 0   name         object 
 1   url          object 
 2   reviews_url  object 
 3   price        float64
 4   item_id      float64
 5   developer    object 
 6   Year         Int64  
 7   genres       object 
 8   tags         object 
 9   specs        object 
dtypes: Int64(1), float64(2), object(7)
memory usage: 152.7+ MB


In [130]:
convert_dict ={
    'item_id'       : 'int32',
    'name'          : 'string',
    'price'         : 'float32',
    'Year'          : 'int32',
    'developer'     : 'category',
    'genres'        : 'category',
    'tags'          : 'category',
    'specs'         : 'category',
    'url'           : 'string',
    'reviews_url'   : 'string'
}
df_combined = df_combined.astype(convert_dict)

In [131]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1977302 entries, 0 to 1977301
Data columns (total 10 columns):
 #   Column       Dtype   
---  ------       -----   
 0   name         string  
 1   url          string  
 2   reviews_url  string  
 3   price        float32 
 4   item_id      int32   
 5   developer    category
 6   Year         int32   
 7   genres       category
 8   tags         category
 9   specs        category
dtypes: category(4), float32(1), int32(2), string(3)
memory usage: 79.5 MB


In [132]:
# Convertimos el archivo en parquet y en csv para el SQL.
# Parquet 
# df_combined.to_parquet("steam_games.parquet")
# CSV
# df_combined.to_csv("steam_games.csv", index=False)

***NLP Machine Learning***

In [1]:
# Importamos librerias 

In [43]:
import re
import nltk
from nltk.stem.porter import PorterStemmer


Normalizamos a los desarrolladores de Ubisoft-Francia, Latam == 'ubisoft'

In [133]:
# Solo queremos ver esta columnas
deve = df_games[['developer']]
# Buscamos todos los desarrolladores que comienzen la letra 'UB'
u_starting_developer_games = deve[deve['developer'].str.startswith('Ub', na=False)]

# Veamos un recuento de cada desarrollador que se encontro
count_d = u_starting_developer_games['developer'].value_counts()
count_d

developer
Ubisoft - San Francisco                                                                                                                      1259
Ubisoft Montreal                                                                                                                               85
Ubisoft                                                                                                                                        45
Ubisoft Quebec, in collaboration with Ubisoft Annecy, Bucharest, Kiev, Montreal, Montpellier, Shanghai, Singapore, Sofia, Toronto studios      10
Ubisoft Montréal                                                                                                                                8
Ubisoft Sofia,Ubisoft Kiev                                                                                                                      6
Ubisoft Montreal, Red Storm, Shanghai, Toronto, Kiev                                                              

In [134]:
# Isntanciamos el poter 
stemmer = PorterStemmer()

def normalize_developer(name):
    name = re.sub(r'[\W_]+', ' ', name) # Reemplazar caracteres especiales y múltiples espacios con un solo espacio
    words = name.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    normalized_name = ' '.join(stemmed_words)
    if 'ubisoft' in normalized_name:
        return 'ubisoft'
    return normalized_name

In [135]:
# aplicamos 
df_combined['normalized_developer']  = df_combined['developer'].apply(normalize_developer)

In [136]:
df_combined.sample(4)

,name,url,reviews_url,price,item_id,developer,Year,genres,tags,specs,normalized_developer
0,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...,4.99,761140,Kotoshiro,2018,Action,Strategy,Single-player,kotoshiro
1,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...,4.99,761140,Kotoshiro,2018,Action,Action,Single-player,kotoshiro
2,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...,4.99,761140,Kotoshiro,2018,Action,Indie,Single-player,kotoshiro
3,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...,4.99,761140,Kotoshiro,2018,Action,Casual,Single-player,kotoshiro


In [137]:
# Verificamos 
deve = df_combined[['normalized_developer']]
# Buscamos todos los desarrolladores que comienzen la letra 'UB'
u_starting_developer_games = deve[deve['normalized_developer'].str.startswith('ub', na=False)]

# Veamos un recuento de cada desarrollador que se encontro
count_d = u_starting_developer_games['normalized_developer'].value_counts()
count_d

normalized_developer
ubisoft                                42487
uber entertain                           900
ubiart montpelli feral interact mac      120
Name: count, dtype: int64

In [138]:
# Cambiamos de dato 
df_combined['normalized_developer'] = df_combined['normalized_developer'].astype('category')

In [139]:
# Elimnamos la columna developer 
df_combined = df_combined.drop(columns=['developer'])

In [140]:
# Cambiamos el nombre a developer 
df_combined = df_combined.rename(columns={'normalized_developer': 'developer'})

In [141]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1977302 entries, 0 to 1977301
Data columns (total 10 columns):
 #   Column       Dtype   
---  ------       -----   
 0   name         string  
 1   url          string  
 2   reviews_url  string  
 3   price        float32 
 4   item_id      int32   
 5   Year         int32   
 6   genres       category
 7   tags         category
 8   specs        category
 9   developer    category
dtypes: category(4), float32(1), int32(2), string(3)
memory usage: 79.5 MB


In [143]:
# Parquet 
# df_combined.to_parquet("steam_games.parquet")